# Introduction to Neural Networks and PyTorch
---
### Outline
* Setup
* Main Components
    * Neural Networks Architecture
        * Multi-Layers Perceptron
            * Linear Layers (a.k.a fully connected layers)
            * Nonlinear Activation Layers
            * Loss function
        * Convolutional Neural Networks (brief introduction)
    * Gradients Calculation with Backpropagation
    * Parameters Optimization
        * Gradient Descent
        * Storchastic Gradient Descent
* Current and Future Reserach 

---

### Setup

In practice, the following dependencies are not needed for this tutorial if you have immediate skills with computer programming. However, to reduce the complexity of this tutorial and take advantages of the speed up provided by parrralalizing and the GPU, we will rely on the folling tools. To learn how to code it with out any of the following dependencies, you can following this [video tutorial](https://www.youtube.com/watch?v=hfMk-kjRv4c).

* **PyTorch** (For creating tensors of various sizes, calculating gradient analytically and performing parameters optimization)
* **matplotlib** (Plotting tool)
* **pandas** (Holding data for plotting)

[Optional] Start by installing [Anaconda](https://www.anaconda.com/products/distribution) or [miniconda](https://docs.conda.io/en/latest/miniconda.html#latest-miniconda-installer-links) to create python enviorment. Otherwise, replace the command `conda` with `pip`

In [ ]:
# Create conda enviorment (optional)
conda create --name pydemo python=3.8

conda activate pydemo

# Pytorch: CPU version (recommendation: use `www.pytorch.org` to choose the best PyTorch version for you)
conda install pytorch cpuonly -c pytorch

# Pytorch: Nvidia GPU version (recommendation: use `www.pytorch.org` to choose the best PyTorch version for you)
conda install pytorch cudatoolkit=11.6 -c pytorch -c conda-forge

conda install matplotlib seaborn -c conda-forge

In [ ]:
import torch as th
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd

#### Setup: *Random Dataset*

The following code is to generate an random dataset with 2 inputs and 1 label for demonstrations

In [ ]:
ndata = 400
layer_n = [2, 3, 3, 1]

## data generation
X = th.rand(ndata, layer_n[0])
Y = 1.0*(th.linalg.norm(X-th.tensor([0.1, 0.1]), dim=1) > 0.5)
X_grid, Y_grid = th.meshgrid(
                th.linspace(0, 1, steps=100), 
                th.linspace(0, 1, steps=100), indexing='ij')
grid_in = th.stack((X_grid, Y_grid), 2).view(-1, 2)

## store data for plotting
dtpd = pd.DataFrame(X, columns=['$x_1$', '$x_2$'])
dtpd['y'] = Y
Y = Y.unsqueeze(dim=-1)
dtpd['label_color'] = dtpd['y'].apply(lambda y: "red" if y==1 else "blue")

## plotting
plt.scatter(data=dtpd, x='$x_1$', y='$x_2$', c=dtpd['label_color'])
plt.show()

In [ ]:
# Training and plot functions

def train(net, opt, criterion, epochs=10000, train_size=-1, batch_size=-1):
    losses = []

    for epoch in range(epochs):  # loop over the dataset multiple times

        # zero the parameter gradients
        opt.zero_grad()

        # forward + backward + optimize
        outputs = net(X[:train_size])
        loss = criterion(outputs, Y[:train_size])
        loss.backward()
        opt.step()

        # log loss
        losses.append(loss.item())

    return losses

def get_heatmap(net):
    heat = net(grid_in).view(100, 100)
    pred = net(X)
    return heat, pred

def plot_boundary_loss(fig, heat, losses, title="Test"):
    heat[heat > 0.51] = 1.0
    heat[heat < 0.49] = 0.0
    heat = heat.cpu().detach().numpy()

    subfgs = fig.subfigures(1, 2)
    ax1 = subfgs[0].subplots(1, 1)
    ax2 = subfgs[1].subplots(1, 1)

    ax1.imshow(heat, cmap='bwr', interpolation='bicubic', alpha=0.4, origin='lower')
    ax1.set_title('Boundary line')
    ax1.scatter(dtpd['$x_1$']*99, dtpd['$x_2$']*99, c=dtpd['label_color'])
    ax1.set_xlim([0, 99])
    ax1.set_ylim([0, 99])

    # plt.figure()
    ax2.set_title('Loss over Epochs')
    ax2.plot(losses)
    ax2.set_xlabel('epoch')
    ax2.set_ylabel('MSE')
    ax2.grid()

    fig.suptitle(title, fontsize=16)


---

### Neural Networks Architecture

Let $\hat{X}$ represent an input domain, $\hat{Y}$ represent an output codomain and $\hat{f}: \hat{X} \rightarrow \hat{Y}$ an unknown function that maps values from the set $\hat{X}$ to $\hat{Y}$. Given subsets $X \subseteq \hat{X}$ and $Y \subseteq \hat{Y}$, can we build a function $f$ that equals $\hat{f}$?

Recents works on *Universal Approximation Theorem* establishes the capabilities of artificial neural networks (in particular Multi-Layer Perceptron) to approximate the mapping between two euclidian spaces given a big enough error of margin $\epsilon > 0$ and number of neurons satisfying, $$\int_{X}\|f(x)-\hat{f}(x)\|dx < \epsilon$$ There is some works discussing the approximation capabilities of neural networks for non-euclidian spaces but the work is ongoing. For further reading, seen footnote below.

<center><img src="imgs\approximation_classes.png" width="500"/></center>

*Reference:* 
* [Universal Approximation Theorem](https://en.wikipedia.org/wiki/Universal_approximation_theorem) (Wikipedia)
* [A Universal Approximation Theorem of Deep Neural Networks for Expressing Probability Distributions](https://proceedings.neurips.cc/paper/2020/hash/2000f6325dfc4fc3201fc45ed01c7a5d-Abstract.html), Lu, Yulong 2020
* [Neural network approximation](https://www.cambridge.org/core/services/aop-cambridge-core/content/view/7077A90FB36D405D903DCC82683B7A48/S0962492921000052a.pdf/neural-network-approximation.pdf), DeVore 2021
* [Deep Neural Network Approximation Theory](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9363169), Elbrächter 2021

### Neural Networks Architecture: *Multi-Layer Perceptron (MLP)*

Multilayer perceptron (or fully-connected neural networks) are a sequence of repeating linear tranformations followed by non-linear transformation for which the linear transform matrixed $W_i$ are learniable parameters.

<center><img src="imgs/nn_structure.bmp" width="500"/></center>

For example, the 3-layers MLP illustrated above can presented the sequence of tranformations:

$$z_1= W_1 x + b_1\\
a_1=\sigma(z_1)\\
z_2= W_2 a_1 + b_2\\
a_2=\sigma(z_2)\\
z_3= W_3 a_2 + b_3\\
h=\sigma(z_3)$$

Which can be condenced to the following equations for batch of inputs and associated outputs:

$$\bar{z}_1= \bar{x}\bar{W_1}^{T}\\
\bar{a}_1=\sigma(\bar{z}_1)\\
\bar{z}_2= \bar{a}_1\bar{W_2}^{T}\\
\bar{a}_2=\sigma(\bar{z}_2)\\
\bar{z}_3= \bar{a}_2\bar{W_3}^{T}\\
\bar{h}=\sigma(\bar{z}_3)$$

Where , $\sigma$ is a non-linear function (e.g. sigmoid function, ReLu, etc.) and

$$\bar{x} = \begin{bmatrix} x_{b1}^T & 1\\ x_{b2}^T & 1\\ \vdots & \vdots \\ x_{bk}^T & 1\end{bmatrix}, 
\bar{W_i} = \begin{bmatrix} W_i & b_i\end{bmatrix}$$

such that $\bar{W}_i \in R^{n\times (m+1)}$ are learnable parameters, $x_{bi}\in \hat{X} \subseteq R^m$, $n$ is the size of the output vector and $m$ is the size of the input vector and k is the size of input batch.

### A MLP without Non-Linear Activation Layers

Let $x_i \in X \subseteq \hat{X} = \{x: 0 \leq x \leq 1, x \in R^2 \}$, $y_i \in Y  \subseteq \hat{Y} = \{0, 1\}$ and $\hat{f}: \hat{X} \rightarrow \hat{Y}$ an unknown function. Then, let $f_{MLP}(x)$ be a neural network that approximates $\hat{f}$ defined as,

$$f_{MLP}(x) = \sigma(\bar{x}\bar{W_1}^{T}\bar{W_2}^{T}\bar{W_3}^{T}) = \sigma(\bar{x}\bar{W}_{1,2,3}^{T})$$

Where $\sigma(x) = \frac{1}{1+e^{-x}}$ (sigmoid function), $W_1 \in R^{10\times 2}$, $W_2 \in R^{10\times 10}$ , $W_1 \in R^{1\times 10}$ 

In [ ]:
## MLP without non-linear activation layers

# Construct model
layers = [nn.Linear(layer_n[i], layer_n[i+1]) for i in range(len(layer_n)-1)]
layers.append(nn.Sigmoid())

model = nn.Sequential(*layers)
criterion = nn.MSELoss()
optimizer = th.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Calculate Gradient and Optimize
losses = train(model, optimizer, criterion)

# Plot boundary line and loss
heat, pred = get_heatmap(model)
fig = plt.figure()
plot_boundary_loss(fig, heat, losses, title="MLP without non-linear functions")
plt.show()


### Non-Linear Activation Layers

As seen in the MLP consisting of just linear activation, a neural network with just linear transformation is not able to create non-linear boundary lines that may separate a given dataset optimally. Thus, we must include non-linear functions to create non-linear boundary lines. 

Let $f_{MLP}(x)$ be a neural network that approximates $\hat{f}$ defined as,

$$f_{MLP}(x) = \sigma(\sigma(\sigma(\bar{x}\bar{W_1}^{T})\bar{W_2}^{T})\bar{W_3}^{T})$$

Where $W_1 \in R^{10\times 2}$, $W_2 \in R^{10\times 10}$, $W_1 \in R^{1\times 10}$ and

$$\sigma(x) = \frac{1}{1+e^{-x}} \text{ (sigmoid function)}$$

In [ ]:
xline = th.linspace(-15, 15, steps=100)
plt.axhline(0, color='grey')
plt.axvline(0, color='grey')
plt.plot(xline, 1/(1+th.exp(xline)), label="$\sigma(x)=\\frac{1}{1+e^{-x}}$", c='blue')
plt.title("Sigmoid Activation")
plt.legend(loc=1)
plt.grid()
plt.show()

In [ ]:
## MLP with sigmoid function

# Construct model
layers = []
for i in range(len(layer_n)-1):
    layers.append(nn.Linear(layer_n[i], layer_n[i+1]))
    layers.append(nn.Sigmoid())

model = nn.Sequential(*layers)
criterion = nn.MSELoss()
optimizer = th.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# Calculate Gradient and Optimize
losses = train(model, optimizer, criterion, epochs=15000)

# Plot boundary line and loss
heat, pred = get_heatmap(model)
fig = plt.figure()
plot_boundary_loss(fig, heat, losses, title="MLP with Sigmoid functions")
plt.show()

Now, for all hidden layers (that is all layers except for the output layer or all layers extracting features from input), lets define,

$$\sigma(x) = \max (0, x) \text{ (ReLu function)}$$

For all hidden layers.

In [ ]:
xline = th.linspace(-15, 15, steps=100)
plt.axhline(0, color='grey')
plt.axvline(0, color='grey')
plt.plot(xline, th.relu(xline), label="$\sigma(x)=\\max(0, x)$", c='blue')
plt.legend(loc=1)
plt.title("ReLu Activation")
plt.grid()
plt.show()

In [ ]:
## MLP with Relu function

# Construct model
layers = [nn.Linear(layer_n[0], layer_n[1])]
for i in range(1,len(layer_n)-1):
    layers.append(nn.ReLU())
    layers.append(nn.Linear(layer_n[i], layer_n[i+1]))
layers.append(nn.Sigmoid())

model = nn.Sequential(*layers)
criterion = nn.MSELoss()
optimizer = th.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Calculate Gradient and Optimize
losses = train(model, optimizer, criterion, epochs=15000)

# Plot boundary line and loss
heat, pred = get_heatmap(model)
fig = plt.figure()
plot_boundary_loss(fig, heat, losses, title="MLP with ReLu functions")
plt.show()

### Loss Function

As aforementioned, the goal with a neural network is to define a function $f(x)$ such that 

$$\int_{X}\|f(x)-\hat{f}(x)\|dx < \epsilon$$ 

for the smallest $\epsilon > 0$ possible. As such, we must optimize our neural network so that $\hat{f}_{obj}(X) = \int_{X}\|f(x)-\hat{f}(x)\|dx$ is as small as possible. However, it has been found that optimizing for $\hat{f}_{obj}(X)$ directly may at times take a significant amount of time and resources. As such, researchers have proposed various objective function alternatives $f_{obj}(X)$ known as loss functions that based on a given unknown function $\hat{f}(x)$, they may converge faster to a function $f(x)$ that has a lower $\epsilon$ than if one was to optimize for $\hat{f}_{obj}(X)$ directly. There are instances that $\hat{Y} \subseteq Y$ may also not be known. Common loss functions include:

1. **MAE, MSE, etc.**
    $$f_{obj}(X,Y) = \sum^{N}_i \frac{ \| f(x_i)- y_i \|}{N} $$
2. **MBE**
    $$f_{obj}(X,Y) = \sum^{N}_i \frac{ f(x_i)- y_i}{N}$$
3. **Cross Entropy Loss/Negative Log Likelihood**
    $$f_{obj}(X,Y) = -\sum^{N}_i [f(x_i) log(y_i) + (1-f(x_i)) log(1 - y_i)]$$
4. **Heat Equation**
    $$f_{obj}(X) = \sum^{N}_i \frac{\partial f}{\partial \bar{x}_t} - c^2 \frac{\partial f}{\partial \bar{x}_x} \bigg\rvert_{\bar{x}=x_i} $$

### Neural Networks Architecture: *Convolutional Neural Network (CNN)*

Although it is not appropriate for the classification task at hand, there is another popular nueral network architecture that we should reference. That is the convolutional neural network. It relies on the convolution operation which is defined as,

$$(f \ast g)(t):=\int_{t_0}^{t_f} f(t-\tau) g(\tau) d \tau$$

is the continous domain, and, more appropriate for us, as,

$$(f \ast g)[n]:= \sum_{k=k_0}^{k_f} f[n-k] g[k]$$

in the discrite domain where [$t_0$, $t_f$] and [$k_0$, $k_f$] is the support of $g(t)$ and $g[k]$, respectively, and $f(t)$ and $f[k]$ are expected to have a larger than or equal to support as $g(t)$ and $g[k]$, respectively. Note that the fourier and laplace tranforms are application of convolution in which the kernel $g(t)$ is define as $e^{-st}$ and $e^{-jwt}$, respectivetly, with free variables s and frequency w. Also, note that these transform hold special properties for convolutions which will not be discussed in this tutorial. Below are a vizualizations of 1D and 2D convolutions, respectively.

    1D Convolution
<center><img src="imgs\1D_Convolution_continous.gif" width="500"/> </center>
<center><img src="imgs\1D_Convolution.gif" height="300"/> </center>

    2D Convolution
<center><img src="imgs\2D_Convolution.gif" height="300"/></center>

Now, the idea with CNN is to make $g[k]$ an discritized tensor $\tilde{g}[k]$ that is a learnable parameter like $W$ (aforementioned during MLP discussion). Then, inserting the convolution operation into an existing neural network structure. For example, an typical CNN would look something like,

$$z_1 = (f \ast \tilde{g})[\tilde{X}]\\
a_1 = \sigma(z_1)\\
z_2 = (f \ast \tilde{g})[a_1]\\
a_2 = \sigma(z_2)\\
a^*_2 = vector(a_2)\\
h = f_{MLP}(a^*_2)$$

where $(f \ast g)[\tilde{X}]$ is the convolution operation of every element of the discritized input $f$ in the domain $\tilde{X}$ (think of an image), $f_{MLP}(x)$ is an MLP as described before, and $vector(a)$ is the operation of flattening the input tensor $a \in R^{n \times \dots \times n}$ into a vector $\hat{a} \in R^{n^k}$ where k is the number of dimensions of $a$.

---

### Gradients Calculation with Backpropagation

Although it is not necessary to optimize the paramaters of a neural network (such as when using evolution based optimization), many optimization schemes use gradients to train neural networks. As such, the backpropagation algorithm was developed to automatically compute the gradients of operations in the neural network and loss function with respect to various variables and parameters. In paritcular, most optimization scheme rely on calculation the greadient of the loss (i.e. object) with repespect to the learnable parameters and using that information to update the model for teh next training iteration. It relies on the chain rule of calculus to avoid recomputing gradients, and solves for the gradients analytacially by tracking the operations performed and storing the known gradient formulation. To explore backpropagation, we will use PyTorch autograd tool and look at the following example.

Given $x_i \in X \subseteq \hat{X}$ and $y_i \in Y \subseteq \hat{Y}$, lets define a generic neural network and a generic associated loss function,

$$f_{MLP}(x)=\sigma(W_2 \sigma(W_1 x + b_1) + b_2)$$

$$f_{obj}(x_i,y_i) = (f_{MLP}(x_i)- y_i)^2 $$

These set of operations can further be discomposed into sequence of simpler operations,

$$
z_1= W_1 x_i + b_1\\
a_1=\sigma(z_1)\\
z_2= W_2 a_1 + b_2\\
h=\sigma(z_2)\\
L = (h - y_i)^2
$$

Now, the goal with this backpropagation algorithm is to calculate the gradient of the loss function $L$ with respect to each of the learnable parameter by starting with the loss and calculating the gradient backwards from the loss to each parameter. For example, to calculate the gradient $\frac{\partial L}{\partial W_1}$ and $\frac{\partial L}{\partial b_2}$, we can discompose them using the chain rule the following ways,

$$\frac{\partial L}{\partial W_1} = \textcolor{grey}{\frac{\partial L}{\partial h} \frac{\partial h}{\partial z_2}} \frac{\partial z_2}{\partial a_1} \frac{\partial a_1}{\partial z_1}  \frac{\partial z_1}{\partial W_1}$$
$$\frac{\partial L}{\partial b_2} = \textcolor{grey}{\frac{\partial L}{\partial h} \frac{\partial h}{\partial z_2}} \frac{\partial z_2}{\partial b_2}$$

As seen, we can take advantage of the fact that many partial derivatives (examples highlighted in $\textcolor{grey}{grey}$) of the gradients of the loss are shared and thus, for a given input, one can store these partial derivatives and reuse them. We can also observe that to calculate the gradients of loss $L$ with respect to an earlier learnable parameters in the order of operations, we must first compute the gradient of the loss with respect to that parameter's function out. Hence, the algorithm name, 'backpropagation' as we must first work backwards to compute the partial derivative of the later operations before we can calculate the earlier ones and propagate the results back.

PyTorch's autograd and others' backpropagation algorithms additionally keeps an record of the analytical partial derivative fomulation of each simplier operation with respect to each of the parameters (learnable and unlearnable ones). This allows the tool to calculate the exact result for each individual gradient. Common partitial derivative formulas include:

$$\frac{\partial (Wx+b)}{\partial W} = x $$

$$\frac{\partial (Wx+b)}{\partial x} = W $$

$$\frac{\partial (Wx+b)}{\partial b} = 1 $$

$$\frac{\partial \sigma}{\partial x} = \frac{e^x}{1+e^{x}} = \sigma(x)(1-\sigma(x)) \text{ (Sigmoid)}$$

$$\frac{\partial \sigma}{\partial x} = \begin{cases} 
0 & \text{if  }  x \leq 0 \\
1 & \text{if  }  x > 0 \\
\end{cases} \text{ (ReLu)}$$

$$\frac{\partial ((h - y_i)^2)}{\partial h} = 2h $$

In [ ]:
# example of pytorch backwards and autograd

x = th.rand(2, requires_grad=True).T
W = th.rand(2, 2, requires_grad=True)
b = th.rand(2, requires_grad=True).T
y = th.ones(2)

z = W*x + b
h = th.sigmoid(z)
loss = h - y

## Method 1: backwards()
loss.backward() # calculate all gradient with respect to loss
print(W.grad)

## Method 2: gradient of h with respect to x
d_h_dx = th.autograd.grad(outputs=h, inputs=x)[0]
print(d_h_dx)

---

### Learnable Parameter Optimization

#### Gradient Decent

#### Storchastic Gradient Decent




In [ ]:
# gradient decent

epochs = 6000
train_size = -1
losses = []

for epoch in range(epochs):

    opt.zero_grad()

    outputs = net(X[:train_size])
    loss = criterion(outputs, Y[:train_size])
    loss.backward()
    opt.step()

    # log loss
    losses.append(loss.item())
